## Data Transformation

This notebook takes the Google BigQuery box score data and transforms it into per-season data.

In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv('game_box_scores.csv' ,low_memory=False)
raw_data.head()

,game_id,season,status,coverage,neutral_site,scheduled_date,gametime,conference_game,tournament,tournament_type,...,a_fast_break_pts,a_second_chance_pts,a_team_turnovers,a_points_off_turnovers,a_team_rebounds,a_flagrant_fouls,a_player_tech_fouls,a_team_tech_fouls,a_coach_tech_fouls,created
0,b4451a02-26c5-4005-9ac8-b06c1f71e661,2015,closed,full,NaN,2015-11-24,2015-11-24 21:30:00+00:00,NaN,NaN,NaN,...,36.0,17.0,0.0,31.0,5.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:58+00:00
1,b2f579ca-9eff-4b2b-a747-81169399c2e8,2015,closed,full,NaN,2015-11-24,2015-11-24 02:00:00+00:00,NaN,NaN,NaN,...,16.0,25.0,0.0,25.0,1.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:53+00:00
2,571be71c-a5bf-446e-bf21-30eb6c54ac5e,2015,closed,full,NaN,2015-11-25,2015-11-25 19:30:00+00:00,NaN,NaN,NaN,...,6.0,6.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:58+00:00
3,d6617923-0b23-49e4-af9b-9e4d0243e45c,2015,closed,full,NaN,2015-12-19,2015-12-19 04:00:00+00:00,NaN,NaN,NaN,...,2.0,13.0,1.0,15.0,3.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:53+00:00
4,ffb463a4-dd3c-4ed9-b503-311b95ef0295,2015,closed,full,NaN,2015-12-20,2015-12-20 04:00:00+00:00,NaN,NaN,NaN,...,6.0,7.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:53+00:00


In [3]:
raw_data.columns.values

array(['game_id', 'season', 'status', 'coverage', 'neutral_site',
       'scheduled_date', 'gametime', 'conference_game', 'tournament',
       'tournament_type', 'tournament_round', 'tournament_game_no',
       'attendance', 'lead_changes', 'times_tied', 'periods',
       'possession_arrow', 'venue_id', 'venue_city', 'venue_state',
       'venue_address', 'venue_zip', 'venue_country', 'venue_name',
       'venue_capacity', 'h_name', 'h_market', 'h_id', 'h_alias',
       'h_league_id', 'h_league_name', 'h_league_alias', 'h_conf_id',
       'h_conf_name', 'h_conf_alias', 'h_division_id', 'h_division_name',
       'h_division_alias', 'h_logo_large', 'h_logo_medium',
       'h_logo_small', 'h_points_game', 'h_rank', 'h_minutes',
       'h_field_goals_made', 'h_field_goals_att', 'h_field_goals_pct',
       'h_three_points_made', 'h_three_points_att', 'h_three_points_pct',
       'h_two_points_made', 'h_two_points_att', 'h_two_points_pct',
       'h_blocked_att', 'h_free_throws_made', 'h_fre

In [4]:
raw_data.tournament_round.unique()

array([nan, 'First Round', 'Second Round', 'Sweet 16', 'Elite Eight',
       '12', 'Championship Series', 'Quarterfinals', 'Championship',
       'Semifinal 2', 'Quarterfinal 2', 'Quarterfinal 1', 'Semifinal 1',
       'Quarterfinal 4', 'Quarterfinal 3', 'Semifinals', 'Quarterfinal',
       'Game 3', 'Game 1', 'Game 2', 'American', 'Game 4', 'Final',
       'Third Round', 'Elite 8', 'Eastern Athletic'], dtype=object)

In [27]:
all_temp_data = raw_data[['season', 'h_market', 'a_market', 'h_id', 'a_id', 'h_points_game', 'a_points_game', 'tournament_round', 'tournament', 'gametime', 'h_conf_name']]

temp_data = all_temp_data[(all_temp_data['tournament_round'] == 'Championship') & (all_temp_data['tournament'] == 'Conference')]
pac_12_data = all_temp_data[(all_temp_data['h_conf_name'] == 'Pacific 12') & (all_temp_data['tournament'] == 'Conference')]
mac_data = all_temp_data[(all_temp_data['h_conf_name'] == 'Mid-American') & (all_temp_data['tournament'] == 'Conference')]

idx = pac_12_data.groupby(['season'])['gametime'].transform(max) == pac_12_data['gametime']
pac_12_data = pac_12_data[idx]

idx = mac_data.groupby(['season'])['gametime'].transform(max) == mac_data['gametime']
mac_data = mac_data[idx]

temp_data = pd.concat([temp_data, pac_12_data, mac_data])

#print(temp_data[temp_data.season == 2016])
h_conf_champ = temp_data[temp_data.h_points_game > temp_data.a_points_game]
a_conf_champ = temp_data[temp_data.h_points_game < temp_data.a_points_game]
 
h_conf_champ = h_conf_champ[['season', 'h_id']]
a_conf_champ = a_conf_champ[['season', 'a_id']]

h_conf_champ.columns = ['season', 'team_id']
a_conf_champ.columns = ['season', 'team_id']

conf_champs = h_conf_champ.append(a_conf_champ)
conf_champs['Conference_Champ'] = True

conf_champs.head()

,season,team_id,Conference_Champ
523,2015,fae4855b-1b64-4b40-a632-9ed345e1e952,True
586,2015,0dadedb0-2bd3-45e7-91a3-93af6c4e87f2,True
600,2015,4743cb7c-784a-4b95-a380-5471f92f2217,True
730,2015,efba5350-79bc-47aa-a189-db177b95b491,True
819,2015,4c9fb59b-6cec-4b0d-bb0f-628b391d138c,True


In [28]:
raw_data['NCAA'] = raw_data.tournament == 'NCAA'
raw_data['Conference'] = raw_data.tournament == 'Conference'
raw_data['ConferenceGame'] = raw_data.h_conf_id == raw_data.a_conf_id

home_team_tournament = raw_data[['season', 'h_id', 'NCAA', 'Conference']]
away_team_tournament = raw_data[['season', 'a_id', 'NCAA', 'Conference']]

home_team_tournament.columns = ['season', 'team_id', 'NCAA', 'Conference']
away_team_tournament.columns = ['season', 'team_id', 'NCAA', 'Conference']

team_tournament = home_team_tournament.append(away_team_tournament)
team_tournament = team_tournament.groupby(['season','team_id']).max().reset_index()

team_tournament.head()

,season,team_id,NCAA,Conference
0,2013,0095032d-6143-44f2-8974-f6815fc56c5b,False,True
1,2013,0113eea0-c943-4fff-9780-ae0fb099e7ef,True,True
2,2013,0163cb22-b8d6-4a98-8ba9-c48a006b27f9,False,False
3,2013,0196c7b3-95a1-4f93-9701-e9013f9c1187,False,False
4,2013,01b0a316-317f-4120-8447-9831231aef0b,False,False


In [29]:
data = raw_data[raw_data.tournament != 'NCAA']

home_team_stats = data[['season', 'h_name', 'h_market', 'h_id', 'h_alias',
       'h_conf_id','h_division_alias',
       'h_conf_name', 'h_conf_alias','h_points_game',
       'h_field_goals_made', 'h_field_goals_att', 'h_field_goals_pct',
       'h_three_points_made', 'h_three_points_att', 'h_three_points_pct',
       'h_two_points_made', 'h_two_points_att', 'h_two_points_pct',
       'h_blocked_att', 'h_free_throws_made', 'h_free_throws_att',
       'h_free_throws_pct', 'h_offensive_rebounds',
       'h_defensive_rebounds', 'h_rebounds', 'h_assists', 'h_turnovers',
       'h_steals', 'h_blocks', 'h_assists_turnover_ratio',
       'h_personal_fouls', 'h_ejections', 'h_foulouts', 'h_points',
       'h_fast_break_pts', 'h_second_chance_pts', 'h_team_turnovers',
       'h_points_off_turnovers', 'h_team_rebounds', 'h_flagrant_fouls',
       'h_player_tech_fouls', 'h_team_tech_fouls', 'h_coach_tech_fouls',
       'a_points_game', 'a_field_goals_pct', 'a_rebounds' ,'a_turnovers']]
away_team_stats = data[['season', 'a_name', 'a_market', 'a_id', 'a_alias',
       'a_conf_id','a_division_alias',  'a_conf_name',
       'a_conf_alias', 'a_points_game', 
       'a_field_goals_made', 'a_field_goals_att', 'a_field_goals_pct',
       'a_three_points_made', 'a_three_points_att', 'a_three_points_pct',
       'a_two_points_made', 'a_two_points_att', 'a_two_points_pct',
       'a_blocked_att', 'a_free_throws_made', 'a_free_throws_att',
       'a_free_throws_pct', 'a_offensive_rebounds',
       'a_defensive_rebounds', 'a_rebounds', 'a_assists', 'a_turnovers',
       'a_steals', 'a_blocks', 'a_assists_turnover_ratio',
       'a_personal_fouls', 'a_ejections', 'a_foulouts', 'a_points',
       'a_fast_break_pts', 'a_second_chance_pts', 'a_team_turnovers',
       'a_points_off_turnovers', 'a_team_rebounds', 'a_flagrant_fouls',
       'a_player_tech_fouls', 'a_team_tech_fouls', 'a_coach_tech_fouls',
       'h_points_game', 'h_field_goals_pct', 'h_rebounds', 'h_turnovers']]

home_team_stats.columns = ['season', 'name', 'market', 'team_id', 'alias',
       'conf_id', 'division_alias',
       'conf_name', 'conf_alias', 'points_game', 
       'field_goals_made', 'field_goals_att', 'field_goals_pct',
       'three_points_made', 'three_points_att', 'three_points_pct',
       'two_points_made', 'two_points_att', 'two_points_pct',
       'blocked_att', 'free_throws_made', 'free_throws_att',
       'free_throws_pct', 'offensive_rebounds',
       'defensive_rebounds', 'rebounds', 'assists', 'turnovers',
       'steals', 'blocks', 'assists_turnover_ratio',
       'personal_fouls', 'ejections', 'foulouts', 'points',
       'fast_break_pts', 'second_chance_pts', 'team_turnovers',
       'points_off_turnovers', 'team_rebounds', 'flagrant_fouls',
       'player_tech_fouls', 'team_tech_fouls', 'coach_tech_fouls',
       'opp_points_game', 'opp_field_goals_pct', 'opp_rebounds', 'opp_turnovers']

away_team_stats.columns = ['season', 'name', 'market', 'team_id', 'alias',
       'conf_id', 'division_alias',
       'conf_name', 'conf_alias', 'points_game',
       'field_goals_made', 'field_goals_att', 'field_goals_pct',
       'three_points_made', 'three_points_att', 'three_points_pct',
       'two_points_made', 'two_points_att', 'two_points_pct',
       'blocked_att', 'free_throws_made', 'free_throws_att',
       'free_throws_pct', 'offensive_rebounds',
       'defensive_rebounds', 'rebounds', 'assists', 'turnovers',
       'steals', 'blocks', 'assists_turnover_ratio',
       'personal_fouls', 'ejections', 'foulouts', 'points',
       'fast_break_pts', 'second_chance_pts', 'team_turnovers',
       'points_off_turnovers', 'team_rebounds', 'flagrant_fouls',
       'player_tech_fouls', 'team_tech_fouls', 'coach_tech_fouls',
       'opp_points_game', 'opp_field_goals_pct', 'opp_rebounds', 'opp_turnovers']

team_stats = home_team_stats.append(away_team_stats)
team_stats = team_stats.groupby(['season', 'name', 'market', 'team_id', 'alias',
       'conf_id', 'division_alias',
       'conf_name', 'conf_alias']).mean().reset_index()

###
### NOTE: There may be instances where we want non-D1 data.
###
team_stats = team_stats[team_stats['division_alias'] == 'D1']

team_stats.head()

,season,name,market,team_id,alias,conf_id,division_alias,conf_name,conf_alias,points_game,...,points_off_turnovers,team_rebounds,flagrant_fouls,player_tech_fouls,team_tech_fouls,coach_tech_fouls,opp_points_game,opp_field_goals_pct,opp_rebounds,opp_turnovers
0,2013,49ers,Charlotte,8ddbfca9-a931-4908-aa31-9fcd17624b5f,CHAR,6902bb03-02f7-4da4-8261-de5fd2cbd011,D1,Conference USA,CUSA,71.451613,...,10.750000,3.0,0.000000,0.75,0.0,0.0,71.645161,41.925,37.5,12.250000
1,2013,49ers,Long Beach State,11a617d5-af03-4cce-bc8a-51114d9e41fe,LBSU,5f56da49-5d81-43c3-8652-8b8da40c907f,D1,Big West,BIGWEST,69.741935,...,10.900000,3.9,0.000000,0.00,0.0,0.0,70.580645,47.450,35.4,11.500000
2,2013,Aces,Evansville,d8217105-e9c2-4267-9538-3aaeea1ae167,EVAN,1d9219c1-696c-42a2-b1be-e8b438795521,D1,Missouri Valley,MVC,69.393939,...,7.666667,4.8,0.166667,0.00,0.0,0.0,72.212121,43.350,38.5,11.166667
4,2013,Aggies,New Mexico State,5016fe1a-9571-4d10-bf5b-b9c1b496bd57,NMSU,7265822b-247a-4991-ae8c-0e6f33f6122b,D1,Western Athletic,WAC,77.228571,...,17.000000,3.5,0.500000,0.00,0.0,0.0,66.914286,50.150,37.0,10.500000
5,2013,Aggies,North Carolina A&T,53aaac6a-796e-41a6-a04e-6ce74c5717f8,NCAT,852139a9-389c-4eca-9760-ec13f014ca8d,D1,Mid Eastern Athletic,MEAC,65.281250,...,3.500000,4.5,0.000000,0.00,0.0,0.0,73.187500,47.450,33.5,9.500000


In [30]:
wins_data = data[['season', 'h_id', 'a_id', 'h_points_game', 'a_points_game', 'ConferenceGame']]

home_wins_data = wins_data.copy()
away_wins_data = wins_data.copy()

home_wins_data['Win'] = home_wins_data.h_points_game > home_wins_data.a_points_game
home_wins_data['Loss'] = home_wins_data.h_points_game < home_wins_data.a_points_game
home_wins_data['Conf_Win'] = home_wins_data.ConferenceGame & (home_wins_data.h_points_game > home_wins_data.a_points_game)
home_wins_data['Conf_Loss'] = home_wins_data.ConferenceGame & (home_wins_data.h_points_game < home_wins_data.a_points_game)

away_wins_data['Win'] = away_wins_data.h_points_game < away_wins_data.a_points_game
away_wins_data['Loss'] = away_wins_data.h_points_game > away_wins_data.a_points_game
away_wins_data['Conf_Win'] = away_wins_data.ConferenceGame & (away_wins_data.h_points_game < away_wins_data.a_points_game)
away_wins_data['Conf_Loss'] = away_wins_data.ConferenceGame & (away_wins_data.h_points_game > away_wins_data.a_points_game)

home_wins_data = home_wins_data[['season', 'h_id', 'Win', 'Loss', 'Conf_Win', 'Conf_Loss']]
home_wins_data.columns = ['season', 'team_id', 'Win', 'Loss', 'Conf_Win', 'Conf_Loss']

away_wins_data = away_wins_data[['season', 'a_id', 'Win', 'Loss', 'Conf_Win', 'Conf_Loss']]
away_wins_data.columns = ['season', 'team_id', 'Win', 'Loss', 'Conf_Win', 'Conf_Loss']

wins_data = home_wins_data.append(away_wins_data)

wins_data = wins_data.groupby(['season', 'team_id']).sum().reset_index()
wins_data['Win_Perc'] = wins_data.Win / (wins_data.Win + wins_data.Loss)
wins_data['Conf_Win_Perc'] = wins_data.Conf_Win / (wins_data.Conf_Win + wins_data.Conf_Loss)
wins_data['Conf_Win_Perc'].fillna(0, inplace=True)

wins_data.head(15)

,season,team_id,Win,Loss,Conf_Win,Conf_Loss,Win_Perc,Conf_Win_Perc
0,2013,0095032d-6143-44f2-8974-f6815fc56c5b,6.0,23.0,4.0,13.0,0.206897,0.235294
1,2013,0113eea0-c943-4fff-9780-ae0fb099e7ef,21.0,12.0,15.0,7.0,0.636364,0.681818
2,2013,0163cb22-b8d6-4a98-8ba9-c48a006b27f9,0.0,1.0,0.0,0.0,0.000000,0.000000
3,2013,0196c7b3-95a1-4f93-9701-e9013f9c1187,0.0,1.0,0.0,0.0,0.000000,0.000000
4,2013,01b0a316-317f-4120-8447-9831231aef0b,0.0,4.0,0.0,0.0,0.000000,0.000000
5,2013,0204a5e6-1228-4ce6-bf3b-3169b46245c0,0.0,1.0,0.0,0.0,0.000000,0.000000
6,2013,02345eb9-071d-4621-a259-a55833bf4391,0.0,2.0,0.0,0.0,0.000000,0.000000
7,2013,03f48445-3994-42bb-a9b1-5c22f98fa5c6,0.0,1.0,0.0,0.0,0.000000,0.000000
8,2013,054c3e85-0552-4549-b123-7e84af6e7b6c,13.0,18.0,4.0,12.0,0.419355,0.250000
9,2013,05b80aaf-a4f0-46f2-b559-e79466056992,0.0,1.0,0.0,0.0,0.000000,0.000000


In [31]:
joined_data = team_stats.merge(team_tournament)
joined_data = joined_data.merge(wins_data)
joined_data = joined_data.merge(conf_champs, how = 'outer')
joined_data['Conference_Champ'].fillna(False, inplace=True)

# Check for duplicates
dup_data = joined_data.groupby(['season', 'team_id']).count()
dup_data = dup_data[dup_data['name'] != 1]

## THIS SHOULD BE EMPTY
print(dup_data.head())

joined_data.head()

Empty DataFrame
Columns: [name, market, alias, conf_id, division_alias, conf_name, conf_alias, points_game, field_goals_made, field_goals_att, field_goals_pct, three_points_made, three_points_att, three_points_pct, two_points_made, two_points_att, two_points_pct, blocked_att, free_throws_made, free_throws_att, free_throws_pct, offensive_rebounds, defensive_rebounds, rebounds, assists, turnovers, steals, blocks, assists_turnover_ratio, personal_fouls, ejections, foulouts, points, fast_break_pts, second_chance_pts, team_turnovers, points_off_turnovers, team_rebounds, flagrant_fouls, player_tech_fouls, team_tech_fouls, coach_tech_fouls, opp_points_game, opp_field_goals_pct, opp_rebounds, opp_turnovers, NCAA, Conference, Win, Loss, Conf_Win, Conf_Loss, Win_Perc, Conf_Win_Perc, Conference_Champ]
Index: []

[0 rows x 55 columns]


,season,name,market,team_id,alias,conf_id,division_alias,conf_name,conf_alias,points_game,...,opp_turnovers,NCAA,Conference,Win,Loss,Conf_Win,Conf_Loss,Win_Perc,Conf_Win_Perc,Conference_Champ
0,2013,49ers,Charlotte,8ddbfca9-a931-4908-aa31-9fcd17624b5f,CHAR,6902bb03-02f7-4da4-8261-de5fd2cbd011,D1,Conference USA,CUSA,71.451613,...,12.250000,False,True,17.0,14.0,7.0,8.0,0.548387,0.466667,False
1,2013,49ers,Long Beach State,11a617d5-af03-4cce-bc8a-51114d9e41fe,LBSU,5f56da49-5d81-43c3-8652-8b8da40c907f,D1,Big West,BIGWEST,69.741935,...,11.500000,False,True,14.0,17.0,10.0,7.0,0.451613,0.588235,False
2,2013,Aces,Evansville,d8217105-e9c2-4267-9538-3aaeea1ae167,EVAN,1d9219c1-696c-42a2-b1be-e8b438795521,D1,Missouri Valley,MVC,69.393939,...,11.166667,False,True,14.0,19.0,8.0,10.0,0.424242,0.444444,False
3,2013,Aggies,New Mexico State,5016fe1a-9571-4d10-bf5b-b9c1b496bd57,NMSU,7265822b-247a-4991-ae8c-0e6f33f6122b,D1,Western Athletic,WAC,77.228571,...,10.500000,True,True,26.0,9.0,13.0,3.0,0.742857,0.812500,True
4,2013,Aggies,North Carolina A&T,53aaac6a-796e-41a6-a04e-6ce74c5717f8,NCAT,852139a9-389c-4eca-9760-ec13f014ca8d,D1,Mid Eastern Athletic,MEAC,65.281250,...,9.500000,False,True,9.0,23.0,5.0,12.0,0.281250,0.294118,False


In [32]:
joined_data.to_csv("agg_team_stats.csv", index = False)